In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras_nlp
import numpy as np
import os

Using TensorFlow backend


In [ ]:
path_tales = 'bajki/'

texts = ''
for index, path in enumerate(os.listdir(path_tales)):
    text = ''
    with open(path_tales + path, encoding='utf-8') as f:
        text = f.read()
        texts += text

In [ ]:
import tqdm
from nltk.tokenize import sent_tokenize, word_tokenize
import string
import re
import nltk
nltk.download('punkt')

def clean_text(txt):
    # Remove punctuation
    txt = "".join(v for v in txt if v not in string.punctuation and '\n' not in v)

    # Remove non-alphanumeric characters (including special characters)
    txt = re.sub(r'[^a-zA-Ząćęłńóśźż\s]', '', txt)

    # Convert to lowercase
    txt = txt.lower()

    return txt

path_tales = 'bajki/'
sentences = []

for filename in tqdm.tqdm(os.listdir(path_tales)):
    with open(path_tales + filename, 'r', encoding='utf-8') as f:
        tale_text = f.read()
        tale_text = sent_tokenize(tale_text)

        sentences.extend([clean_text(l) for l in tale_text])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
100%|██████████| 85/85 [00:12<00:00,  6.82it/s]


In [ ]:
sentences[:10]

['pewna uboga wdowa mieszkała w samotnej chatce a przed tą chatką był ogródek w którym rosły dwa krzaki róż jedne róże białe drugie czerwone',
 'wdowa miała dwie córeczki podobne do obu krzaków róż',
 'jedna zwała się białośnieżką a druga różanką',
 'dziewczątka były tak pobożne i dobre tak pracowite i roztropne że chyba drugich takich dzieci nie było na świecie ale białośnieżka była łagodniejszą niż różanka',
 'różanka wolała skakać po łąkach i polach rwała kwiatki i goniła ptaszyny leśne a białośnieżka siadywała w domu przy matce pomagała jej w gospodarstwie albo czytywała jej książki gdy nic innego nie było do roboty',
 'obie dziewczynki tak bardzo się kochały że zawsze idąc razem trzymały się za rączki a gdy białośnieżka mówiła my się nigdy nie rozstaniemy różanka odpowiadała tym co ma jedna musi się i druga podzielićnieraz biegały same po lesie i zbierały czerwone jagody żadne zwierzę nie robiło im nic złego ale przychodziły do nich z całym zaufaniem zajączek jadł z ich rąk kapust

In [ ]:
import random
random.shuffle(sentences)

In [ ]:
from collections import Counter

# Tokenize the sentences and count word frequencies
word_counts = Counter()
for sentence in sentences:
    words = sentence.split()
    word_counts.update(words)

# Select the top 50,000 most frequent words
top_words = set(word for word, _ in word_counts.most_common(50000))

# Replace the sentences with only the top words
new_sentences = []
for sentence in sentences:
    words = sentence.split()
    selected_words = [word for word in words if word in top_words]
    new_sentences.append(" ".join(selected_words))

In [ ]:
new_sentences[:10]

['widzę nasze chorągwie',
 'mogłeś pan sam odgadnąć',
 'kto przypadł ten sam sobie do woli czerpał z kadzi bo w taki dzień każdy gościem był miłym',
 'zali wytrzymamy do godziny modlitw o głodzie i kto nas później pożywi',
 'osiołek nasz dobrze się miewa',
 'fortuna i inne nie mogło w żaden sposób np',
 'o tym co było zewnątrz ciebie ludzie czcili lecz twoje owoce',
 'więc mury klasztorne naokół jak boki okrętu otoczonego burzą i rozbójnikami',
 'sam doktor ma z tobą na pieńku powiadał niewdzięczny kiedy wspominał ciebie',
 'nel zachwycała się szczególnie widokiem i czarnych sporych ptaków które odzywały się głosem']

In [ ]:
length = len(new_sentences)
text_train = new_sentences[:int(0.7*length)]
text_test = new_sentences[int(0.7*length):int(0.85*length)]
text_valid = new_sentences[int(0.85*length):]

In [ ]:
from tensorflow.keras.layers import TextVectorization

def custom_standardization(input_string):
    sentence = tf.strings.lower(input_string)
    sentence = tf.strings.regex_replace(sentence, "\n", " ")
    return sentence

maxlen = 50
# You can also set calculate the longest sentence in the data - 25 in this case
# maxlen = len(max(text_list).split(' '))

vectorize_layer = TextVectorization(
    standardize = custom_standardization,
    output_mode="int",
    output_sequence_length=maxlen + 1,
)

vectorize_layer.adapt(new_sentences)
vocab = vectorize_layer.get_vocabulary()

In [ ]:
vocab_size = len(vocab)
vocab_size

50002

In [ ]:
index_lookup = dict(zip(range(len(vocab)), vocab))
index_lookup[5]

'nie'

In [ ]:
vectorize_layer(['dawno dawno temu'])


<tf.Tensor: shape=(1, 51), dtype=int64, numpy=
array([[923, 923, 323,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])>

In [ ]:
batch_size = 64

train_dataset = tf.data.Dataset.from_tensor_slices(text_train)
train_dataset = train_dataset.shuffle(buffer_size=256)
train_dataset = train_dataset.batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices(text_test)
test_dataset = test_dataset.shuffle(buffer_size=256)
test_dataset = test_dataset.batch(batch_size)

valid_dataset = tf.data.Dataset.from_tensor_slices(text_valid)
valid_dataset = valid_dataset.shuffle(buffer_size=256)
valid_dataset = valid_dataset.batch(batch_size)

In [ ]:
def preprocess_text(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_dataset = train_dataset.map(preprocess_text)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

test_dataset = test_dataset.map(preprocess_text)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

valid_dataset = valid_dataset.map(preprocess_text)
valid_dataset = valid_dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:
for entry in train_dataset.take(1):
    print(entry)

(<tf.Tensor: shape=(64, 50), dtype=int64, numpy=
array([[   97,  1507,     3, ...,     0,     0,     0],
       [  157,    23,    78, ...,     0,     0,     0],
       [   31,    86,  1738, ...,     0,     0,     0],
       ...,
       [ 1558,    90,   469, ...,     0,     0,     0],
       [  393,  1179,    42, ...,     0,     0,     0],
       [   17,   383, 22407, ...,     0,     0,     0]])>, <tf.Tensor: shape=(64, 50), dtype=int64, numpy=
array([[ 1507,     3,     8, ...,     0,     0,     0],
       [   23,    78,  6898, ...,     0,     0,     0],
       [   86,  1738,    11, ...,     0,     0,     0],
       ...,
       [   90,   469, 12951, ...,     0,     0,     0],
       [ 1179,    42,   441, ...,     0,     0,     0],
       [  383, 22407,   818, ...,     0,     0,     0]])>)


In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras_nlp

embed_dim = 128
num_heads = 4

def create_model():
    inputs = keras.layers.Input(shape=(maxlen,), dtype=tf.int32, name='input_layer')
    embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(vocab_size, maxlen, embed_dim, name='embedding_layer')(inputs)
    decoder = keras_nlp.layers.TransformerDecoder(intermediate_dim=embed_dim,
                                                  num_heads=num_heads,
                                                  dropout=0.5, name='transformer_decoder')(embedding_layer)

    outputs = keras.layers.Dense(vocab_size, activation='softmax', name='output_layer')(decoder)

    model = keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer="adam",
        loss='sparse_categorical_crossentropy',
        metrics=[keras_nlp.metrics.Perplexity(), 'accuracy']
    )
    return model

model = create_model()
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 50)]              0         
                                                                 
 embedding_layer (TokenAndP  (None, 50, 128)           6406656   
 ositionEmbedding)                                               
                                                                 
 transformer_decoder (Trans  (None, 50, 128)           99584     
 formerDecoder)                                                  
                                                                 
 output_layer (Dense)        (None, 50, 50002)         6450258   
                                                                 
Total params: 12956498 (49.43 MB)
Trainable params: 12956498 (49.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
import random
class TextSampler(keras.callbacks.Callback):
    def __init__(self, start_prompt, max_tokens):
        self.start_prompt = start_prompt
        self.max_tokens = max_tokens

    # Helper method to choose a word from the top K probable words with respect to their probabilities
    # in a sequence
    def sample_token(self, logits):
        logits, indices = tf.math.top_k(logits, k=5, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)

    def on_epoch_end(self, epoch, logs=None):
        decoded_sample = self.start_prompt

        for i in range(self.max_tokens-1):
            tokenized_prompt = vectorize_layer([decoded_sample])[:, :-1]
            predictions = self.model.predict([tokenized_prompt], verbose=0)
            # To find the index of the next word in the prediction array.
            # The tokenized prompt is already shorter than the original decoded sample
            # by one, len(decoded_sample.split()) is two words ahead - so we remove 1 to get
            # the next word in the sequence
            sample_index = len(decoded_sample.strip().split())-1

            sampled_token = self.sample_token(predictions[0][sample_index])
            sampled_token = index_lookup[sampled_token]
            decoded_sample += " " + sampled_token

        print(f"\nSample text:\n{decoded_sample}...\n")

# First 5 words of a random sentence to be used as a seed
seed_sentence = 'dawno dawno temu'
sampler = TextSampler(seed_sentence, 50)
reducelr = keras.callbacks.ReduceLROnPlateau(patience=10, monitor='val_loss')
checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath='checkpoints/transformer_{epoch:02d}')


In [ ]:
model = create_model()
history = model.fit(train_dataset,
                    validation_data=valid_dataset,
                    epochs=100,
                    callbacks=[sampler, reducelr, checkpoint_callback])

Epoch 1/100
2672/2672 [==============================] - ETA: 0s - loss: 1.9391 - perplexity: 6.9524 - accuracy: 0.7705
Sample text:
dawno dawno temu nie mógł  już z sobą  do nich  w której by mu się na ziemię z nim się do nich  i z nim na świecie ...

2672/2672 [==============================] - 605s 223ms/step - loss: 1.9391 - perplexity: 6.9524 - accuracy: 0.7705 - val_loss: 1.7457 - val_perplexity: 5.7299 - val_accuracy: 0.7753 - lr: 0.0010
Epoch 2/100
2672/2672 [==============================] - ETA: 0s - loss: 1.6517 - perplexity: 5.2160 - accuracy: 0.7767
Sample text:
dawno dawno temu temu i z tego nie ma się do niego  w niej w tej nocy gdy mu w drogę  i  tak jest to że to jest ...

2672/2672 [==============================] - 548s 205ms/step - loss: 1.6517 - perplexity: 5.2160 - accuracy: 0.7767 - val_loss: 1.7189 - val_perplexity: 5.5783 - val_accuracy: 0.7755 - lr: 0.0010
Epoch 3/100
2672/2672 [==============================] - ETA: 0s - loss: 1.5376 - perplexity: 4.6535 - ac